# SAPA for ScF3 - Grouped Modes

In this example, we will run SAPA for X-ray PDF data for ScF3, refining all modes for each irrep together. 

ScF3 is a formed from corner-sharing ScF6 octahedra, so shares a lot of similarities to the perovskite structure, such that the mode listing for ScF3 is a subset of those found for the ABX3 perovskite.

Firstly, we need to tell the python kernel where to look for the sapa scripts. These are in the directory a level above this one, so can be done straightforwardly using the sys module:

In [10]:
import sys
sys.path.append('..')

Now, we can import the main sapa script, and create a SAPA object. You can use the already downloaded CIF to do this, or create one yourself. ScF3 has a Pm-3m space group, with F at [$1/2, 0, 0$] and Sc at either [$1/2, 1/2, 1/2$] or [$0, 0, 0$]. Note that the different origin choices will result in different irrep labels. The pre-made "ScF3_iso.cif" using the [$1/2, 1/2, 1/2$] setting.

In [2]:
import sapa as s

struc = s.sapa('ScF3_iso.cif')

The SAPA object has all the variables contained in the CIF. Anything defined with loop_ in the CIF will be a list in Python, otherwise it is a single variable. By default, everything is imported as a string; numbers can be converted to floats using the float() function.

There are several functions that act on the information imported from the CIF. The most basic is:

In [3]:
struc.irrep_list()

['GM4-',
 'R5+',
 'X1+',
 'X5+',
 'M2-',
 'M5-',
 'GM5-',
 'R2-',
 'R3-',
 'R4-',
 'R5-',
 'X2+',
 'X3-',
 'X5-',
 'M1+',
 'M2+',
 'M3+',
 'M4+',
 'M5+']

Which prints out a list of irreps defined within the CIF. The remaining functions we will go through step-by-step, in this and further tutorials. The first is the write_inp() method; this writes the TOPAS input file for the grouped mode method of operation. It takes several arguments, some of which have default values:

* sample : This is a string, used to name the output files and to create the monitoring file. It should contain information                such as the material being studied, e.g. for a first run of ScF3, sample = 'ScF3' should suffice. This can be                  changed by editing the input file after creation.

* filenameformat : This is a string to allow for looping over the PDF data files. For SAPA, you should make sure your data                        files have a consistent name format, e.g. for this tutorial, all files are of the format                                        'ScF3_tempK_dofr.xye', where temp is the temperature the data was collected at. I can then supply a list of                    temperatures to the execute command. For the filenameformat variable, the thing you are replacing each time                    (temp for this example) should be replaced by ##n##. A macro in TOPAS replaces 'n' with the temperature you                    want to execute, and the two ## tell the TOPAS interpreter to join the temperature to the rest of the                          string.

* isneutron : False if using X-ray data, True if using neutron - note these are not strings, but booleans; the capital letters               are important

* qmax : The maximum value of Q used to generate the PDF data. This is a float.

* dq : The resolution in Q-space. This should be obtained by using a standard. This is a float.

* startx, finishx : The minimum and maximum radius for the calculated PDF. Both floats.

* lattice : Designates the lattice parameters to be refined, depending on the lattice type. Options are 'C', 'T', 'O', 'H',                'R', 'M' and 'TC'

* path : The folder containing the data files. By default, this is the working directory. 

* cycles : The number of repetitions to perform for each irrep for each data file. By default, this is 300. This is a float.

* bin : The bin size used in the PDF data. By default, this is 0.02. This is a string.

* lor : The lorentzian contribution to dQ damping - by default, this is 0. This is a float.

* alpha : Alpha parameter for Q-dependent broadening. Note - this is default 0, since including it slows down refinements                 considerably. This is a float.

* filename : The filename for the created input file. By default, this is 'batch_modes.inp'. This requires a .inp extension.                This is a string.

* refine_angles : False by default; set to True if you want to refine angle lattice parameters.

* version : "7" by default. Set to "6" if you use TOPAS version 6, which requires a couple extra lines to be added to the input             file.

The data for this tutorial was collected at beamline P02.1 at PETRA III synchrotron, DESY. It was generated with a Qmax of 21.0 and a dQ of 0.08. Since the deviations of the local structure from the average will be most evident at low r, where atomic motion is most correlated, I recommend an r-max of between 1-2x the supercell lattice vector. ScF3 is cubic throughout the entire studied temperature range. There was only a minor lorentzian contribution to the peak shape, so this was set to 0. Alpha was also set to 0. Using this information, the write_inp() command looks like this:

In [4]:
struc.write_inp(sample = 'ScF3',
               filenameformat = 'ScF3_##n##K_dofr.xye',
               isneutron = False,
               qmax = 21.0,
               dq = 0.08,
               startx = 1.0,
               finishx = 10.0,
               lattice = 'C')

# Note 1: This assumes you use Topas version 7. If you use version 6, add an additional argument of version='6'. 
# If you have a version of Topas prior to 6, then this does not include PDF fitting.

# Note2: These are all the arguments without defaults. 
# If you wish to change the filename from its default value, or reduce the number of cycles, you can add these as additional arguments.

# Note3: The format of the command here is done for clarity. It can all be entered on one line, separated by commas.

# Note4: This is the correct order for the arguments. If you specify them in this order, you do not have to write argument =
# i.e the command could be struc.write_inp('ScF3','ScF3_###n##K_dofr.xye',False,21.0,0.08,1.0,10.0,'C')
# you do have to specify the variable name for changes to default values, e.g. the above command with reducing the number of cycles to 100
# could look like struc.write_inp('ScF3','ScF3_###n##K_dofr.xye',False,21.0,0.08,1.0,10.0,'C', cycles=100)



Creating input file...
96
96
...batch_modes.inp written


When creating the input file, the script writes the number of modes to the console. In most cases (i.e. looking at displacive modes for all atoms in the input CIF), this will be 3x(supercell volume/input cell volume)x(atoms in input cell), e.g. for ScF3 we have 4 atoms in the input cell, the supercell volume is 8x bigger than the input cell (2^3 = 8), giving us 96 modes. The filename of the written input file is also printed.

Now we have created an input file, we can execute SAPA. This is done with the aptly named "execute" function. This function creates a text file with a list of the irrep/temp combinations supplied to it, and tracks whether this combination has been executed yet. This allows the run to be restarted, in case something interrupted SAPA, or if you need to terminate the run if it has 'hung' (i.e its been a while since a new irrep/temp combo has been ran). This can happen if you lock your PC, or let it go to sleep. It is recommended to edit your power settings so that the PC and Screen never go to sleep (search power and sleep settings).

Before executing SAPA, we need to create a list of temperatures for it to iterate over. Note that while these are the temperatures the data were collected at, Topas doesn't know this, and we are just supplying it part of a filename. These also don't have to be temperatures, this is just what the variable is called internally within the SAPA code; they could be dopant levels, pressures or some other variable. 



In [5]:
temps = ['125', '140', '147', '152', '175', '200', '225', '250', '275', '325', '350', '375', '400']

# Note1: These are defined as a list due to the irregular interval between temperatures. If, for your own data, you have regular steps
# you can use the range(start,stop+1,step) function to define temps. You can add ranges together; e.g. if you collected data from
# 100 - 300 K in 25K steps, but from 300 - 400 K you used 10 K steps, you could use temps = range(100,301,25) + range(300,401,10)

For standard operation, the execute function only needs one argument; the above defined temps. The function itself searches for the Topas executables, so you do not need to supply a Topas directory.



In [14]:
struc.execute(temps)

Finding Topas executables...
Enter filename: batch_modes
Enter sample name for this run: ScF3
Creating Monitoring File...
Executing...
Executing irrep GM4- for temp 125
Completed irrep GM4- for temp 125. Process took 200.72981214523315s (3.3454968690872193 minutes)
Estimated 14.441394818226497 hours to completion.
Executing irrep R5+ for temp 125
Completed irrep R5+ for temp 125. Process took 114.10494589805603s (1.9017490983009337 minutes)
Estimated 11.281578829884529 hours to completion.
Executing irrep X1+ for temp 125
Completed irrep X1+ for temp 125. Process took 158.28727436065674s (2.638121239344279 minutes)
Estimated 11.258552067390196 hours to completion.
Executing irrep X5+ for temp 125
Completed irrep X5+ for temp 125. Process took 301.7367856502533s (5.028946427504222 minutes)
Estimated 13.775267876519097 hours to completion.
Executing irrep M2- for temp 125
Completed irrep M2- for temp 125. Process took 152.6031777858734s (2.5433862964312235 minutes)
Estimated 13.139044941

Completed irrep GM5- for temp 147. Process took 89.19278264045715s (1.4865463773409526 minutes)
Estimated 8.732692758056288 hours to completion.
Executing irrep R2- for temp 147
Completed irrep R2- for temp 147. Process took 116.45595645904541s (1.940932607650757 minutes)
Estimated 8.653491443837131 hours to completion.
Executing irrep R3- for temp 147
Completed irrep R3- for temp 147. Process took 115.36838364601135s (1.9228063941001892 minutes)
Estimated 8.57490157908458 hours to completion.
Executing irrep R4- for temp 147
Completed irrep R4- for temp 147. Process took 97.91093564033508s (1.631848927338918 minutes)
Estimated 8.477806415319444 hours to completion.
Executing irrep R5- for temp 147
Completed irrep R5- for temp 147. Process took 92.27905011177063s (1.5379841685295106 minutes)
Estimated 8.377041312732727 hours to completion.
Executing irrep X2+ for temp 147
Completed irrep X2+ for temp 147. Process took 90.7237753868103s (1.512062923113505 minutes)
Estimated 8.2774378206

Completed irrep X5- for temp 175. Process took 97.30693769454956s (1.6217822949091594 minutes)
Estimated 6.520898673408983 hours to completion.
Executing irrep M1+ for temp 175
Completed irrep M1+ for temp 175. Process took 117.85972595214844s (1.9643287658691406 minutes)
Estimated 6.470250761331174 hours to completion.
Executing irrep M2+ for temp 175
Completed irrep M2+ for temp 175. Process took 141.736563205719s (2.36227605342865 minutes)
Estimated 6.431306399093183 hours to completion.
Executing irrep M3+ for temp 175
Completed irrep M3+ for temp 175. Process took 94.81631660461426s (1.5802719434102377 minutes)
Estimated 6.370451745004452 hours to completion.
Executing irrep M4+ for temp 175
Completed irrep M4+ for temp 175. Process took 131.30572414398193s (2.1884287357330323 minutes)
Estimated 6.327056855364722 hours to completion.
Executing irrep M5+ for temp 175
Completed irrep M5+ for temp 175. Process took 95.81760573387146s (1.5969600955645242 minutes)
Estimated 6.267770396

Completed irrep GM4- for temp 250. Process took 192.17985916137695s (3.202997652689616 minutes)
Estimated 4.607084132125756 hours to completion.
Executing irrep R5+ for temp 250
Completed irrep R5+ for temp 250. Process took 123.73389482498169s (2.0622315804163613 minutes)
Estimated 4.564758949057784 hours to completion.
Executing irrep X1+ for temp 250
Completed irrep X1+ for temp 250. Process took 201.89066815376282s (3.3648444692293804 minutes)
Estimated 4.540307922959328 hours to completion.
Executing irrep X5+ for temp 250
Completed irrep X5+ for temp 250. Process took 335.6625266075134s (5.594375443458557 minutes)
Estimated 4.545351112972439 hours to completion.
Executing irrep M2- for temp 250
Completed irrep M2- for temp 250. Process took 147.4117865562439s (2.456863109270732 minutes)
Estimated 4.5075658142361155 hours to completion.
Executing irrep M5- for temp 250
Completed irrep M5- for temp 250. Process took 265.6533260345459s (4.427555433909098 minutes)
Estimated 4.4953831

Completed irrep GM5- for temp 325. Process took 82.40280294418335s (1.3733800490697226 minutes)
Estimated 2.855764168496296 hours to completion.
Executing irrep R2- for temp 325
Completed irrep R2- for temp 325. Process took 85.08812856674194s (1.4181354761123657 minutes)
Estimated 2.8107138904358475 hours to completion.
Executing irrep R3- for temp 325
Completed irrep R3- for temp 325. Process took 118.01118183135986s (1.9668530305226644 minutes)
Estimated 2.7693257560066304 hours to completion.
Executing irrep R4- for temp 325
Completed irrep R4- for temp 325. Process took 84.65869188308716s (1.4109781980514526 minutes)
Estimated 2.724614957699999 hours to completion.
Executing irrep R5- for temp 325
Completed irrep R5- for temp 325. Process took 84.79777002334595s (1.4132961670557658 minutes)
Estimated 2.6801400476361237 hours to completion.
Executing irrep X2+ for temp 325
Completed irrep X2+ for temp 325. Process took 89.60846042633057s (1.4934743404388429 minutes)
Estimated 2.636

Completed irrep X3- for temp 375. Process took 102.23229026794434s (1.7038715044657389 minutes)
Estimated 1.0363517104607285 hours to completion.
Executing irrep X5- for temp 375
Completed irrep X5- for temp 375. Process took 86.81144142150879s (1.4468573570251464 minutes)
Estimated 0.9963828573992223 hours to completion.
Executing irrep M1+ for temp 375
Completed irrep M1+ for temp 375. Process took 99.0938720703125s (1.6515645345052083 minutes)
Estimated 0.9569118927838389 hours to completion.
Executing irrep M2+ for temp 375
Completed irrep M2+ for temp 375. Process took 100.19950294494629s (1.6699917157491049 minutes)
Estimated 0.9175733885522616 hours to completion.
Executing irrep M3+ for temp 375
Completed irrep M3+ for temp 375. Process took 126.73298788070679s (2.1122164646784465 minutes)
Estimated 0.8790472476330377 hours to completion.
Executing irrep M4+ for temp 375
Completed irrep M4+ for temp 375. Process took 117.97908282279968s (1.9663180470466615 minutes)
Estimated 0.

By default, the execute function suppresses Topas output, as the full progress of the least-squares refinement gets printed to the console. Instead, it calculates a rough estimate of the length of time remaining (based on the average time per refinement up to this point, meaning it should get more accurate as time goes on). This can be changed by adding the argument verbose = True as the first commented command below. I recommend you do this the first time you run SAPA for your own data, as it prints any Topas errors to the terminal window. 

If something did happen to either terminate your SAPA run early, or make it hang, you can stop the execute command in this notebook by pressing the square stop button at the top of the page, or in a command prompt Ctrl+C. You can start again from where you stopped it by using the commented restart command below.

If your python environment was closed during a run (e.g. by the bane of overnight runs, the IT department forcing your PC to restart), then you will need to make a SAPA object again, and then restart. The restart function will ask you to input a sample name and file name. 

In [7]:
#struc.execute(temps, verbose = True)
#struc.restart()

## After the SAPA run has finished

SAPA will take a few hours to finish in the grouped mode operation, so in this tutorial I recommend to run the execute function overnight, and return to this part of the tutorial in the morning. 

When the SAPA run has finished, you will notice that it has created a large number of text files (3 for each irrep / temp combination). Sometimes, Topas can have a minor error (one that stops an irrep/temp combo before it has reached the cycle limit, but doesn't stop the entire script from running), which means some of the files could be missing, or not at the correct size. The next function checks if all the files are there and the correct size, and re-runs and irrep/temp combos where this is not the case

In [6]:
struc.file_check(temps, verbose = False)

# If you have changed the number of cycles in the input file, then this will also need to specified as an argument here. 


Files checked, found no issues!


This can be run multiple times, until it prints 'Files checked, found no issues'.

The large number of text files produced is cumbersome to work with. To simplify this, the next two scripts store all the information contained within them in a single file in an HDF5 format, then cleans up the text files. The HDF5 files have a consistent structure, and are easily interactable with Python (see later tutorials).

In [7]:
struc.create_hdf5(temps)

struc.cleanup()

# By default, cleanup creates a zip file containing all the text files. If you would prefer to just delete them, then add 
# create_zip = False as an argument.

Creating zip file...
...Zip file written.
Deleting .txt files...
...Clean up complete.


This is the end of the "execution" segment of using SAPA. From here, move on to further tutorials; GroupMode_ScF3_Analysis for analysis of this data or SingleMode_ScF3 for executing the single mode version of SAPA.